# Daily Tweet Processing
Iterate tweets for a given day and perform some calculations

In [1]:
# required imports to access api_db, misc, misc.CONFIG, ...
import sys
sys.path = ['.', '..', '../..'] + sys.path
from collection import *

using 105 seed accounts (20 from news sources) and 0 hashtags
Database stats: 
{'avgObjSize': 81.86696522187272,
 'collections': 3,
 'dataSize': 4606736.0,
 'db': 'electionswatch',
 'fsTotalSize': 510770802688.0,
 'fsUsedSize': 445133578240.0,
 'indexSize': 970752.0,
 'indexes': 3,
 'numExtents': 0,
 'objects': 56271,
 'ok': 1.0,
 'scaleFactor': 1.0,
 'storageSize': 2363392.0,
 'views': 0}
DB size (B): 4606736.0
DB size (MB): 4.39
DB size (GB): 0.00
Using API keys for app twitta-mos

Done initializing at 01:07AM on September 25, 2020.
----------------------------------------


<hr>
<h1 align="center">driver code</h1>

1. Define tasks to execute daily
2. Check if these have been executed for each day of the past month
3. Execute the ones that have not yet been executed (this ensures that new ones will be retroactively updated)
4. 

---
Define tasks below - these should be replicated in the api (or maybe make it generably able to access any `task_%s` collection from any endpoint, assuming injection is not a concern atm)

In [3]:
# task 1 - count tweets by type
from collections import defaultdict
def count_tweets_by_type(day):
    task = Task(api_db.db, "count by type")
    if task.exists_day(day): return # already processed
    metrics = defaultdict(int, next(api_db.col_tweets.aggregate([
        {"$match": {"created_at": get_filter_by_day(day)}}, 
        {"$facet": {
            "retweet": [
                {"$match": {"retweeted_status": {"$exists": True}}},
                {"$count": "retweet"},
            ],
            "quote": [
                {"$match": {"quoted_status": {"$exists": True}}},
                {"$count": "quote"},
            ],
            "reply": [
                {"$match": {"in_reply_to_status_id": {"$exists": True}}},
                {"$count": "reply"},
            ],
            "original": [
                {"$match": {"original": True}},
                {"$count": "original"},
            ],
            "total": [
                {"$count": "total"},
            ]
        }},
        {"$project": {
            "retweet": {"$arrayElemAt": ["$retweet.retweet", 0]},
            "quote": {"$arrayElemAt": ["$quote.quote", 0]},
            "reply": {"$arrayElemAt": ["$reply.reply", 0]},
            "original": {"$arrayElemAt": ["$original.original", 0]},
            "total": {"$arrayElemAt": ["$total.total", 0]}
        }}
    ])))
    # force defaultdict to create 0
    [metrics[x] for x in ["retweet", "quote", "reply", "original", "total"]]
    task.insert(day, metrics)

# Main function that calls declared tasks
Each task must be manually registered

In [2]:
from datetime import datetime, timedelta

In [4]:
def main_caller(day):
    with DoneMessage("Processing day %s" % day):
        count_tweets_by_type(day)

In [5]:
for day in (misc.CONFIG["collection"]["oldest_tweet"] + timedelta(n) for n in range(31)):
    main_caller(day)

Processing day 2020-09-01 00:00:00...Done in 0.023s.
Processing day 2020-09-02 00:00:00...Done in 0.012s.
Processing day 2020-09-03 00:00:00...Done in 0.012s.
Processing day 2020-09-04 00:00:00...Done in 0.009s.
Processing day 2020-09-05 00:00:00...Done in 0.007s.
Processing day 2020-09-06 00:00:00...Done in 0.011s.
Processing day 2020-09-07 00:00:00...Done in 0.011s.
Processing day 2020-09-08 00:00:00...Done in 0.011s.
Processing day 2020-09-09 00:00:00...Done in 0.010s.
Processing day 2020-09-10 00:00:00...Done in 0.010s.
Processing day 2020-09-11 00:00:00...Done in 0.009s.
Processing day 2020-09-12 00:00:00...Done in 0.008s.
Processing day 2020-09-13 00:00:00...Done in 0.011s.
Processing day 2020-09-14 00:00:00...Done in 0.008s.
Processing day 2020-09-15 00:00:00...Done in 0.009s.
Processing day 2020-09-16 00:00:00...Done in 0.010s.
Processing day 2020-09-17 00:00:00...Done in 0.011s.
Processing day 2020-09-18 00:00:00...Done in 0.013s.
Processing day 2020-09-19 00:00:00...Done in 0

In [ ]:
# print(get_account_details(20509689))

In [ ]:
# from datetime import timezone
# find_params = find_exclude_invalid({
#     "depth": 0
# })
# oldest_t = misc.CONFIG["collection"]["oldest_tweet"]
# # users = api_db.col_users.find(find_params, no_cursor_timeout=True)
# users = [{"_id": 8665852}]
# for u in users:
#     print("getting tweets for: %s..." % u["_id"], end="", flush=True)
#     tweets = get_tweets(u, api_db.api.GetUserTimeline, "since_id",  datetime(2020, 7, 18, tzinfo=timezone.utc), {"trim_user":True})
#     # insert_tweets(tweets)
#     # update_most_common_language(u, tweets) # removed because not useful here
#     print("got %d new tweets, done." % len(tweets))
#     if len(tweets) > 0: break

In [ ]:
# # [print(str(t) + "\n") for t in tweets[-10:-1]]
# for t in tweets[0:10000]:
#     if "in_reply_to_user_id" in t:
#         print(t)

In [ ]:
misc.CONFIG["collection"]["oldest_tweet"]

In [ ]:

for tweet in api_db.col_tweets.find({"created_at": {"$gte": _from, "$lt": _to}}, no_cursor_timeout=True):
    print(tweet)
    break

In [ ]:
print("DONE")